# Librerías

In [1]:
import ctypes
import os
from ctypes import util
import pandas as pd
import numpy as np
from ctypes import *
from ctypes import c_float, POINTER, byref
from itertools import product
import itertools
import pypsa
import time
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
from datetime import timedelta
import gurobipy
from IPython.display import display

In [2]:
import logging
import os

# Suppress all logs from PyPSA and Linopy
logging.getLogger("pypsa").setLevel(logging.CRITICAL)
logging.getLogger("linopy").setLevel(logging.CRITICAL)
logging.getLogger().setLevel(logging.CRITICAL)

# Redirect all Gurobi messages to null
os.environ["GUROBI_LOGFILE"] = "NUL"

# Funciones DLL 
Se carga el archivo DLL con la función load_library y el modelo con load_model. Se corre el modelo con run_model y se obtiene el valor de las variables mediante get_value y se establecen valores mediante set_value

In [3]:
# LOAD THE DLL FILE
def load_vensim_library():
    return load_library()

def load_library():
    dll_name = 'vendll64'
    dll_path = ctypes.util.find_library(dll_name)
    
    if dll_path is None:
        raise FileNotFoundError(f"The DLL {dll_name} could not be found.")
    
    vensim_dll = ctypes.CDLL(dll_path)
    return vensim_dll
    
vensim_dll=load_vensim_library()

In [4]:
# LOAD THE MODEL

def load_model(model_path, vensim_dll):
    command = f'SPECIAL>LOADMODEL|{model_path}'
    result = vensim_dll.vensim_command(command.encode('utf-8'))
    
    if result == 0: 
        raise Exception("Failed to load the Vensim model.")
    
    vensim_dll.vensim_be_quiet(0)   # 0: no suprression (default, dialogs and messages)
                                    #1: suppresses warnings and some dialogs (still error messages)
                                    #2: suppresses all interactions (including error messages)

In [5]:
def cmd(command):
    """Execute a Vensim command and handle errors."""
    result = vensim_dll.vensim_command(command.encode('utf-8'))
    if result == 0:
        raise Exception(f"Command failed: {command}")
def run_model(runname, interval=1):
    """
    Run the Vensim model in game mode with step-by-step control.

    Parameters
    ----------
    runname : str
        Name for the simulation run.
    interval : int
        Number of time steps to advance in each interval.
    """
    cmd(f"SIMULATE>RUNNAME|{runname}")
    cmd("MENU>GAME|O")
    cmd(f"GAME>GAMEINTERVAL|{interval}")

    initial_time = get_value("INITIAL TIME")
    final_time = get_value("FINAL TIME")
    print(f"Running from {initial_time} to {final_time}")

    for t in np.arange(initial_time, final_time, interval):
        run_udfs(t)

    #     # Advance the model to the next step
    #     cmd("GAME>GAMEON")
    # cmd("GAME>ENDGAME")
    # print(f"Simulation '{runname}' completed.")

In [6]:
def get_value(var_name):
    """Retrieve the value of a variable from the Vensim model."""
    result = c_float()
    success = vensim_dll.vensim_get_val(var_name.encode('utf-8'), byref(result))
    if not success:
        raise KeyError(f"Unable to retrieve value for '{var_name}'")
    return result.value

In [7]:
def run_udfs(current_time):
    """Execute user-defined functions at each interval."""
    # print(f"Executing UDFs at time {current_time}")
    # try:
    #     # Retrieve and print a variable value
    #     demand = get_value("Demand by Zone [f0, H19, Z1]")
    #     print(f"Demand at time {current_time}: {demand}")
    # except KeyError as e:
    #     print(f"Warning: {e}")

In [8]:
def set_value(var_name, value):
    command = f'SIMULATE>SETVAL|{var_name}={value}'
    result = vensim_dll.vensim_command(command.encode('utf-8'))
    if result == 0:
        raise Exception(f"Failed to set value for '{var_name}'.")
    # else:
        # print(f"Value set successfully for '{var_name}' = {value}")

# Cargar y correr el modelo
Aquí se establece la ruta del modelo (file_path), el nombre de la corrida (runname) y el intervalo de la corrida (interval).

In [9]:
file_path = "ModeloPol2.vpmx"
load_model(file_path, vensim_dll)
runname='ModPol2'
interval=0.25
run_model(runname,interval=interval)

Running from 0.0 to 195.0


# Definición de parámetros
Se define el vector de ztech (todas las tecnologías de todas las zonas), pueden omitirse generadores y no afecta el código.
En este caso no se incluye Other2, Other3 y PV+BESS

In [10]:
# Complete
ztech=['Z1Biomass','Z1WindOnS', 'Z1WindOffS', 'Z1PV', 'Z1HRoR', 'Z1HydroDam', 'Z1Thermo1', 'Z1Thermo2', 'Z1Geothermal', 'Z1PVBESSOp1', 'Z1PVBESSOp2', 
       'Z2Biomass', 'Z2WindOnS', 'Z2WindOffS', 'Z2PV', 'Z2HRoR', 'Z2HydroDam', 'Z2Thermo1', 'Z2Thermo2', 'Z2Geothermal', 'Z2PVBESSOp1', 'Z2PVBESSOp2',
        'Z3Biomass', 'Z3WindOnS', 'Z3WindOffS', 'Z3PV', 'Z3HRoR', 'Z3HydroDam', 'Z3Thermo1', 'Z3Thermo2', 'Z3Geothermal', 'Z3PVBESSOp1', 'Z3PVBESSOp2',
        'Z4Biomass', 'Z4WindOnS', 'Z4WindOffS', 'Z4PV', 'Z4HRoR', 'Z4HydroDam', 'Z4Thermo1', 'Z4Thermo2', 'Z4Geothermal', 'Z4PVBESSOp1', 'Z4PVBESSOp2', 
        'Z5Biomass', 'Z5WindOnS', 'Z5WindOffS', 'Z5PV', 'Z5HRoR', 'Z5HydroDam', 'Z5Thermo1', 'Z5Thermo2', 'Z5Geothermal', 'Z5PVBESSOp1', 'Z5PVBESSOp2']
# zone=['Caribe', 'Antioquia', 'Nordeste', 'Oriental', 'Suroccidental', 'NN']

In [11]:
# #withouth Other2, Other3 and PV+BESS. Just zones in use
#ztech=['Z1Biomass', 'Z1WindOnS', 'Z1WindOffS', 'Z1PV', 'Z1HRoR', 'Z1HydroDam', 'Z1Thermo1', 'Z1Thermo2', 'Z1Geothermal', 'Z2Biomass', 'Z2WindOnS', 'Z2WindOffS', 'Z2PV', 'Z2HRoR', 'Z2HydroDam', 'Z2Thermo1', 'Z2Thermo2', 'Z2Geothermal', 'Z3Biomass', 'Z3WindOnS', 'Z3WindOffS', 'Z3PV', 'Z3HRoR', 'Z3HydroDam', 'Z3Thermo1', 'Z3Thermo2', 'Z3Geothermal','Z4Biomass', 'Z4WindOnS', 'Z4WindOffS', 'Z4PV', 'Z4HRoR', 'Z4HydroDam', 'Z4Thermo1', 'Z4Thermo2', 'Z4Geothermal', 'Z5Biomass', 'Z5WindOnS', 'Z5WindOffS', 'Z5PV', 'Z5HRoR', 'Z5HydroDam', 'Z5Thermo1', 'Z5Thermo2', 'Z5Geothermal']
zone=['Caribe', 'Antioquia', 'Nordeste', 'Oriental', 'Suroccidental']

In [12]:
# All forecast years
# forecastyear = ['f0','f1','f3','f5','f7','f9']
forecastyear = ['f0','f7']
#forecastyear = ['f0']

In [13]:
start = pd.Timestamp('2025-01-01 00:00:00')  # Simulation start date

# Funciones para: Demand by Zone, Available Generation y Offer Price
Según las funciones definidas con el DLL, se usan para obtener los valores de las variables de Vensim que se usarán para el despacho

In [14]:
def calc_date(start, step):
    curr_time = get_value("Time")
    total_months = curr_time * step
    int_months = int(total_months)
    frac_days = (total_months - int_months) * 30
    
    curr_date = start + relativedelta(months=int_months) + timedelta(days=frac_days)
    curr_date = curr_date.replace(hour=0, minute=0, second=0, microsecond=0)
    return curr_date

In [15]:
def demand_df(year, start_date, zone):
    demand = {f'Z{j}': [] for j in range(1, len(zone)+1)}
    # Only process the specific year passed as parameter, NOT the global forecastyear list
    for hour in [f'H{i}' for i in range(24)]:
        row = []
        for zone_key in demand.keys():
            name = f"{'Residual Demand by zone'} [{year}, {hour}, {zone_key}]"
            try:
                value = get_value(name)
            except KeyError:
                value = None
            row.append(value)
        for i, zone_key in enumerate(demand.keys()):
            demand[zone_key].append(row[i])
    time_index = pd.date_range(start=start_date, periods=24, freq='h')
    # Create multi-index with only the specific year, not all forecast years
    multi_index = pd.MultiIndex.from_product([time_index, [year]], names=['Time', 'ForecastYear'])
    return pd.DataFrame(demand, index=multi_index)

In [16]:
def availability_df(ztech, start_date):
    availability = {tech: [] for tech in ztech}
    for hour in range(24):
        for tech in ztech:
            name = f"Availability Factor[H{hour},{tech}]"
            try:
                value = get_value(name)
            except KeyError as e:
                print(f"Warning: {e}")
                value = None
            availability[tech].append(value)
    time_idx = pd.date_range(start=start_date, periods=24, freq='H')
    return pd.DataFrame(availability, index=time_idx)

In [17]:
def expected_gen(ztech, year):
    gen = {}
    # Only process the specific year passed as parameter, NOT the global forecastyear list
    for tech in ztech:
        name = f"Expected Generation Capacity[{year},{tech}]"
        try:
            value = get_value(name)
        except KeyError as e:
            print(f"Warning: {e}")
            value = None
        gen[tech] = value
    # Return DataFrame with only the specific year as index
    return pd.DataFrame(gen, index=[year])

In [18]:
def offer_price_df(ztech):
    price = pd.DataFrame(columns=ztech, index=[0])
    for tech in ztech:
        name = f"Offer price hydro adjusted[{tech}]"
        try:
            value = get_value(name)
            price.loc[0, tech] = value
        except KeyError as e:
            price.loc[0, tech] = None
            print(f"Warning: {e}")
    return price

In [19]:
def get_int(forecastyear):
    
    lines = [
        "Int1", "Int2", "Int6", "Int10", "Int7", "Int8", "Int13"
    ]

    inter_df = pd.DataFrame(columns=lines, index=forecastyear)

    for year in forecastyear:
        for int_name in lines:
            var_name = f"Expected Interconnection Capacity[{year},{int_name}]" 
            try:
                value = get_value(var_name)  # Function to get values
                inter_df.loc[year, int_name] = value
            except KeyError:
                inter_df.loc[year, int_name] = None
                print(f"Warning: Missing capacity for {int_name} in year {year}")

    return inter_df

# Función para hacer el despacho en PyPSA
Se genera la función para la optimización lineal del despacho en PyPSA con las variables definidas anteriormente

In [20]:
def pypsa_dispatch_lines(current_date, zone, demand_zone, expected_generation, availability_factor, offer_price):
    networks = {}
    dispatch_results= {}
    line_capacity_results = {}

    # Use the years from the actual data passed, not the global forecastyear list
    active_years = expected_generation.index.tolist()

    for year in active_years:
        n = pypsa.Network()
        snapshots = pd.date_range(current_date, periods=24, freq='h')
        n.set_snapshots(snapshots)
        
        bus_data = pd.DataFrame({'name': zone})
        for _, row in bus_data.iterrows():
            n.add("Bus", name=row['name'])
        
        lines = [
            ("Int1", "Antioquia", "Caribe", 250000),
            ("Int2", "Caribe", "Nordeste", 100000),
            ("Int6", "Nordeste", "Antioquia", 160000),
            ("Int10", "Nordeste", "Oriental", 40000),
            ("Int7", "Antioquia", "Oriental", 90000),
            ("Int8", "Antioquia", "Suroccidental", 170000),
            ("Int13", "Suroccidental", "Oriental", 130000),
        ]
            
        for line_name, bus0, bus1, s_nom in lines:
            n.add("Line", line_name, bus0=bus0, bus1=bus1, s_nom=s_nom, x=0.1)
            
            forecast_demand = demand_zone.xs(year, level='ForecastYear')
            
        for i, zone_name in enumerate(forecast_demand.columns):
            bus_name = zone[i]
                
            if bus_name not in n.buses.index:
                raise ValueError(f"Bus '{bus_name}' not found in the network.")
                
            load_series = forecast_demand[zone_name]
            n.add("Load", f"Load_{zone_name}", bus=bus_name, p_set=load_series)
            
        for tech in expected_generation.columns:
            bus_prefix = tech[:2].lower()
            bus_name = zone[int(bus_prefix[1:]) - 1]
            p_nom = expected_generation.loc[year, tech]
            p_max_pu = availability_factor[tech]
            marginal_cost = offer_price.loc[0, tech]
                
            n.add(
                "Generator",
                tech,
                bus=bus_name,
                carrier=tech,
                p_nom=p_nom,
                p_max_pu=p_max_pu.values,
                marginal_cost=marginal_cost,
                p_nom_extendable=False,
                )
        
        n.consistency_check()
        n.optimize(
    solver_name="gurobi",
    solver_options={
        "LogToConsole": 0,
        "OutputFlag": 0,
    }
)
        
        dispatch = n.generators_t.p.transpose().round(2)
        dispatch_results[year] = dispatch

        lines_capacity = n.lines_t.p0.abs().max()
        line_capacity_results[year] = lines_capacity

    return dispatch_results, line_capacity_results

In [21]:
def pypsa_dispatch_congestions(current_date, zone, demand_zone, expected_generation, availability_factor, offer_price, interconnections):
    networks = {}
    dispatch_results= {}
    line_capacity_results = {}
    dispatch_costs_results={}

    lines = {
    "Int1": ("Antioquia", "Caribe"),
    "Int2": ("Caribe", "Nordeste"),
    "Int6": ("Nordeste", "Antioquia"),
    "Int10": ("Nordeste", "Oriental"),
    "Int7": ("Antioquia", "Oriental"),
    "Int8": ("Antioquia", "Suroccidental"),
    "Int13": ("Suroccidental", "Oriental"),
}

    # Use the years from the actual data passed, not the global forecastyear list
    active_years = expected_generation.index.tolist()

    for year in active_years:
        n1 = pypsa.Network()

        snapshots = pd.date_range(current_date, periods=24, freq='h')
        n1.set_snapshots(snapshots)
        
        bus_data = pd.DataFrame({'name': zone})
        for _, row in bus_data.iterrows():
            n1.add("Bus", name=row['name'])

        for int_name, (bus0, bus1) in lines.items():
            capacity = interconnections.loc[year, int_name]
            n1.add("Line", int_name, bus0=bus0, bus1=bus1, s_nom=capacity, x=0.1)
            
            forecast_demand = demand_zone.xs(year, level='ForecastYear')
            
        for i, zone_name in enumerate(forecast_demand.columns):
            bus_name = zone[i]
                
            if bus_name not in n1.buses.index:
                raise ValueError(f"Bus '{bus_name}' not found in the network.")
                
            load_series = forecast_demand[zone_name]
            n1.add("Load", f"Load_{zone_name}", bus=bus_name, p_set=load_series)
            
        for tech in expected_generation.columns:
            bus_prefix = tech[:2].lower()
            bus_name = zone[int(bus_prefix[1:]) - 1]
            p_nom = expected_generation.loc[year, tech]
            p_max_pu = availability_factor[tech]
            marginal_cost = offer_price.loc[0, tech]
                
            n1.add(
                "Generator",
                tech,
                bus=bus_name,
                carrier=tech,
                p_nom=p_nom,
                p_max_pu=p_max_pu.values,
                marginal_cost=marginal_cost,
                p_nom_extendable=False,
                )
        
        n1.consistency_check()
        n1.optimize(
    solver_name="gurobi",
    solver_options={
        "LogToConsole": 0,
        "OutputFlag": 0,
    })
        
        dispatch = n1.generators_t.p.transpose().round(2)
        dispatch_results[year] = dispatch

        dispatch_costs=n1.buses_t.marginal_price.max(axis=1).to_frame().T
        dispatch_costs_results[year]=dispatch_costs
 
    return dispatch_results, dispatch_costs_results

# Función para set_val del despacho
Se crea una función para devolver el valor del despacho a Vensim según la optimización lineal

In [22]:
def set_dispatch_lines(dispatch_results):
   for year in dispatch_results:
       dispatch = dispatch_results[year]
       for ztech in dispatch.index:
        for i, col in enumerate(dispatch.columns):
            hour = f"H{i}"
            name = f"Max Interconnection Dispatch[{year},{ztech},{hour}]"
            val = dispatch.loc[ztech, col]
            set_value(name, val)

def set_dispatch_congestions(dispatch_results):
   for year in dispatch_results:
       dispatch = dispatch_results[year]
       for ztech in dispatch.index:
        for i, col in enumerate(dispatch.columns):
            hour = f"H{i}"
            name = f"Congested Dispatch[{year},{ztech},{hour}]"
            val = dispatch.loc[ztech, col]
            set_value(name, val)

In [23]:
def set_lines(line_capacity_results):
    for year in line_capacity_results:
        capacities = line_capacity_results[year]
        
        for line_name, cap in capacities.items():
            name = f"Max Interconnection Capacity[{line_name},{year}]"
            
            try:
                set_value(name, cap)
            except Exception as e:
                print(f"Failed to set {name} with value {cap}. Error: {e}")

In [24]:
def set_dispatch_costs(dispatch_costs_results):
    for year in dispatch_costs_results:
        costs_df = dispatch_costs_results[year]
       
        for i, col in enumerate(costs_df.columns):
            hour = f"H{i}"
            name = f"Congested Dispatch Price[{year},{hour}]"
            val = costs_df.iloc[0, i]
           
            try:
                set_value(name, val)
            except Exception as e:
                print(f"Failed to set {name} with value {val}. Error: {e}")

# Ejecutar todo
Iteración para todos los pasos de tiempo. En este caso 311 meses, 4 veces cada mes (por eso el intervalo de 0.25)

In [25]:
time_step_interval = get_value("TIME STEP")
print(f"Time Step: {time_step_interval}")
final_time = get_value("FINAL TIME")
print(f"Final time: {final_time}")

Time Step: 0.25
Final time: 195.0


In [26]:
#time_steps = np.arange(0, final_time, time_step_interval)
time_steps = np.arange(0, 195, 0.25)

# Store results dynamically for all forecast years
scenario_results = {}
for year in forecastyear:
    scenario_results[year] = {'dispatch_lines': {}, 'dispatch_congestions': {}, 'line_capacities': {}, 'dispatch_costs': {}}

for curr_time in time_steps:
    current_time = get_value("Time")
    print(f"\n--------------SIMULATION TIME STEP:{current_time}--------------")

    current_date = calc_date(start=start, step=time_step_interval)
    
    # Run both planning scenarios and compare results
    for scenario in forecastyear: 
               
        # Calculate demand for the current scenario
        demand_zone = demand_df(year=scenario, start_date=current_date, zone=zone)

         # Calculate availability factors (same for both scenarios)
        availability_factor = availability_df(ztech=ztech, start_date=current_date)
            
        # Retrieve expected generation data for the scenario
        expected_generation = expected_gen(ztech=ztech, year=scenario)
        
        # Get offer prices (same for both scenarios)
        offer_price = offer_price_df(ztech=ztech)
        if scenario=="f7":    
            print(f"--- PERFORMING DISPATCH FOR MAX INTERCONNECTION CAPACITY ({scenario}) ---")
            # Perform PyPSA dispatch for Max Interconnection Capacity
            dispatch1, line_capacity_results = pypsa_dispatch_lines(
                current_date=current_date,
                zone=zone,
                demand_zone=demand_zone,
                expected_generation=expected_generation,
                availability_factor=availability_factor,
                offer_price=offer_price,
            )
            
            # Store results for comparison
            #scenario_results[scenario]['dispatch_lines'][current_time] = dispatch1
            scenario_results[scenario]['line_capacities'][current_time] = line_capacity_results
            
            # Set values in Vensim for this scenario
            #set_dispatch_lines(dispatch1)
            set_lines(line_capacity_results) 

        # Get interconnections for the scenario
        if scenario=="f0": 
            interconnections = get_int([scenario])
            print(f"--- PERFORMING CONGESTIONS DISPATCH ({scenario}) ---")
            
            # Perform PyPSA dispatch for Congested Dispatch
            dispatch, dispatch_costs = pypsa_dispatch_congestions(
                current_date=current_date,
                zone=zone,
                demand_zone=demand_zone,
                expected_generation=expected_generation,
                availability_factor=availability_factor,
                offer_price=offer_price, 
                interconnections=interconnections,
            )
            
            # Store results for comparison
            scenario_results[scenario]['dispatch_congestions'][current_time] = dispatch
            #scenario_results[scenario]['dispatch_costs'][current_time] = dispatch_costs
            
            # Set values in Vensim for this scenario
            set_dispatch_congestions(dispatch)
            #set_dispatch_costs(dispatch_costs)
            
    # Advance simulation time step
    cmd("GAME>GAMEON")
    print("--------------SIMULATION STEP ADVANCED--------------")
    # break

cmd("GAME>ENDGAME")
print("--------------GAME IS OVER--------------")

--------------SIMULATION STEP ADVANCED--------------

--------------SIMULATION TIME STEP:23.0--------------
--- PERFORMING CONGESTIONS DISPATCH (f0) ---


C:\Users\joan.fuentes\AppData\Local\Temp\ipykernel_16248\4166254967.py:12: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_idx = pd.date_range(start=start_date, periods=24, freq='H')


KeyboardInterrupt: 